# E2 - TP - Programmation concurrente en python

## 1- Processus concurrents.  
    
Comprenez et exécutez le programme python suivant :

In [10]:
from os import getpid #importation de getpid
pid = str(getpid()) #getpid renvoie en si=ortie le pid du processus dans le lequel on se trouve
with open("testPID.txt", "w") as fichier: #ouvre le fihier testPID.txt en écriture (w)
         for i in range (1000):
            fichier.write(pid + " : " + str(i) + "\n")
            fichier.flush() #force l'écriture immédiate de write sur le disque

<div class="alert alert-info">
Le fichier testPID.text a été créé dans le dossir de ce notebook. Ouvrez ce fichier avec notepad++ et dites ce que l'on y trouve :  
    
*Votre réponse ici...*  
On y trouve le PID du processus qui a incrémenté chaque valeur de i.

<div class="alert alert-info">
Vous allez devoir travailler sous linux afin de lancer simultanément 3 instances de python (créant ainsi trois processus simultanés de python lançant ce même script).
    Ouvrez votre repertoire personnel.  
    Clic droit, puis "Ouvrir un terminal ici"  
    Dans votre terminal entrez la command <code>touch testPID.py</code>  
    Puis la commande <code>atom testPID.py</code>  
    Dans atom, copiez le code du programme puis enregistrez et fermez atom.  
    Dans votre terminal,testez votre code en entrant la command <code>python3 testPID.py</code>  
    Regardez le contenu de votre fichier testPID.txt avec la commande <code>atom testPID.txt</code> puis fermez atom.
    Entrez la commande <code>python3 testPID.py & python3 testPID.py & python3 testPID.py &</code> afin de lancer 3 instances de ce script python simultanément.
    Que contient maintenant testPID.txt et comment expliquez vous le contenu de votre fichier testPID.txt?  (pour vous aider, rappelez vous que la méthode write initialise un curseur d'écriture lors de l'ouverture d'un fichier)
    
*Votre réponse ici*  
On a l'impression que certaines valeurs ont été écrites par un processus et d'autres valeurs par un autre processus... En fait an cas de commutation de contexte le programme qui est interrompu a une certaine valeur du curseur d'écriture. L'autre processus va avancer le curseur pour écrire lorsqu'il a la main... puis lorsque le premier v areprendre la main il va reprendre à partir de SA valeur de ciurseur et écraser ce que l'autre processus a écrit... c'est le dernier passé sur une position donnée qui a raison...

## 2- Problématiques d'interblocage et multithreads


Un thread est un sou-processus démarré par un processus et s'exécutant de manière concurrente avec le reste du programme. Le module threading de la bibliohèque standard de Python permet de démarrer des threads.  
Le programme suivant permet d'illustrer ce propos:

In [30]:
import threading

def hello(n):
    for i in range(5):
        print("Je suis le thread ", n, " et ma valeur est ", i)
    print("------ Fin du thread ", n)

for n in range(4):
    t = threading.Thread(target=hello, args=[n]) #la variable t contient l'objet thread créé
    t.start() #On lance l'exécution en tâche de fond et on rend la main au script... la boucle continue

Je suis le thread  0  et ma valeur est  0
Je suis le thread  0Je suis le thread   1  et ma valeur est  0
Je suis le thread  1  et ma valeur est  1
Je suis le thread  1  et ma valeur est  2
Je suis le thread  1  et ma valeur est  3
Je suis le thread  1  et ma valeur est  4
------ Fin du thread  1
 et ma valeur est  1
Je suis le thread  0  et ma valeur est  2
Je suis le thread  0  et ma valeur est  3
Je suis le thread  0  et ma valeur est  4
------ Fin du thread  0
Je suis le thread  2  et ma valeur est  0
Je suis le thread  2  et ma valeur est  1
Je suis le thread  2  et ma valeur est  2Je suis le thread  3  et ma valeur est  0
Je suis le thread  3  et ma valeur est  1
Je suis le thread  3  et ma valeur est 
 2
Je suis le thread  3  et ma valeur est  3
Je suis le thread  3  et ma valeur est  4
------ Fin du thread  3
Je suis le thread  2  et ma valeur est  3
Je suis le thread  2  et ma valeur est  4
------ Fin du thread  2


<div class="alert alert-info">
Vous pouvez exécuter le code ci-dessus plusieurs fois. Obtenez vous toujours le même résultat?  
    
*Votre réponse ici*  
Non...

<div class="alert alert-info">
Comment expliquez vous ce résultat ?  
  
*Votre réponse ici*  
Les threads alternent leur exécution au gré des commutations de contexte.

<div class="alert alert-info">
Quand vous lancez le programme ci dessus, combien de threads sont exécutés simultanément?  
  
*Votre réponse ici*  
5 ! le thread principal de ce programme + les 4 lancés par start.

In [31]:
import threading
COMPTEUR = 0

def incrc():
    '''Cette fonction incrémente la variable globale COMPTEUR de 100000'''
    global COMPTEUR #désolé pour ce coup là mais necessaire pour la démonstration.
    for c in range(100000):
        v = COMPTEUR
        COMPTEUR = v+1
th = [] #On déclare un tableau vide.

for n in range(4):
    t = threading.Thread(target=incrc,args=[]) #la variable t contient l'objet thread créé
    t.start() #On lance l'exécution en tâche d efond et on rend la main au script
    th.append(t) #Après le démarrage du thread, l'objet est stocké dans le tableau th
    
for t in th:
    t.join() #La méthode join permet d'attendre que le thread auquel on l'applique soit terminée.
print("Valeur finale de COMPTEUR : ", COMPTEUR)

Valeur finale de COMPTEUR :  359603


<div class="alert alert-info">
En théorie, on execute 4 threads qui vont augmenter COMPTEUR de 100000 chacun. A quelle valeur de COMPTEUR doit-on logiquement arriver en fin d'exécution ?  
  
*Votre réponse ici*  
400000

<div class="alert alert-info">
Executez plusieurs fois ce script. Quel résultat obtenez vous ?  
  
*Votre réponse ici*  
On obtient au maximum 400000 ou moins...

<div class="alert alert-info">
Comment expliquez vous ce résultat?  
  
*Votre réponse ici*  
Si une commutation de contexte a lieu pour un thread juste après v = COMPTEUR, les autres threads vont augmenter COMPTEUR pendant leur exécution.Mais quand le premier thread reprendra la mais, il réduira la valeur de COMPTEUR (qui avait avancé avec les autres threads) car son v avait enregistré une valeur de COMPTEUR plus faible quelà où il en est.... COMPTEUR recule quand ceci se passe et on obtient une valeur inférieure à 400000 à la fin...

<div class="alert alert-info">
Pour corriger le problème il nous faut éviter d'être coupé en commutation de contexte au beau milieu de <code>v = COMPTEUR</code> et <code>COMPTEUR = v + 1</code> grâce à un verrou qui permetra d'avoir un accès EXCLUSIF à la variable compteur entre le moment de la lecture et celui de son écriture.  
Réécrivez le code ci-dessus en ajoutant la ligne <code>verrou = threading.Lock()</code> après l'import de threading, puis en ajoutant la ligne <code>verrou.acquire()</code> avant <code>v = COMPTEUR</code> et la ligne <code>verrou.release()</code> après la ligne <code>COMPTEUR = v + 1</code>.  
Un verrou est un objet que l'onpeut essayer d'acquérir. Si on est le premier à faire cette demande, on acquiert le verrou. On peut le rendre à tout moment. En revanche si quelqu'un d'autre tient le verrou, alors on est bloqué jusqu'à ce qu'il soit libéré.  
    
Une portion de code protégée par un verrou est appelée SECTION CRITIQUE.
Testez la solution ci dessous:

In [1]:
#Votre code ici
import threading
verrou = threading.Lock()
COMPTEUR = 0

def incrc():
    '''Cette fonction incrémente la variable globale COMPTEUR de 100000'''
    global COMPTEUR #désolé pour ce coup là mais necessaire pour la démonstration.
    for c in range(100000):
        verrou.acquire()
        v = COMPTEUR
        COMPTEUR = v+1
        verrou.release()
th = [] #On déclare un tableau vide.

for n in range(4):
    t = threading.Thread(target=incrc,args=[]) #la variable t contient l'objet thread créé
    t.start() #On lance l'exécution en tâche d efond et on rend la main au script
    th.append(t) #Après le démarrage du thread, l'objet est stocké dans le tableau th
    
for t in th:
    t.join() #La méthode join permet d'attendre que le thread auquel on l'applique soit terminée.
print("Valeur finale de COMPTEUR : ", COMPTEUR)

Valeur finale de COMPTEUR :  400000


<div class="alert alert-info">
Que pensez vous de ce code suivant ? (éxécutez le plusieurs fois)  
    
*Votre réponse ici*  
On a un risque d'interblocage.

In [33]:
import threading

verrou1 = threading.Lock()
verrou2 = threading.Lock()

def f1():
    verrou1.acquire()
    print("Section critique 1.1")
    verrou2.acquire()
    print("Section critique 1.2")
    verrou2.release()
    verrou1.release()
    
def f2():
    verrou2.acquire()
    print("Section critique 2.1")
    verrou1.acquire()
    print("Section critique 2.2")
    verrou1.release()
    verrou2.release()

t1 = threading.Thread(target=f1, args=[])
t2 = threading.Thread(target=f2, args=[])

t1.start()
t2.start()

Section critique 1.1Section critique 2.1

